In [ ]:
from github import Github
import pandas as pd 
import itertools
import json

In [ ]:
%store -r access_token
g = Github("")

In [ ]:
g.rate_limiting

In [ ]:
# KPI 1 > Total number of repositories
repo = g.get_organization("carpentries-incubator").get_repos()
repo.totalCount


In [ ]:
# with open('CarpentriesIncubator.json', 'w', encoding='utf-8') as f:
#     json.dump(repo, f, ensure_ascii=False, indent=4)
#      with open("Data.json", "r") as read_file:
#     variable = json.load(read_file)

In [ ]:
for i in range(0,10):
    if len(repo.get_page(i))==0:
        print(i, "======================Empty")
        break

    else:
        pass
        #print(repo.get_page(i))

In [ ]:
repo_list = list()
for i in range(0,5):
    repo_list.append(repo.get_page(i))
    

In [ ]:
len(repo_list)

In [ ]:
repo_list

In [ ]:
final_repo_list = list((itertools.chain.from_iterable(repo_list)))

In [ ]:
len(final_repo_list)

In [ ]:
p=10
print(final_repo_list[p].full_name)

In [ ]:
# This program give two Repository lists. 1) Repository _list_with_lesson & Repository _list_without_lesson.

list_without_lesson=list()
list_with_lesson=list()
for repo in final_repo_list:
  tags=repo.get_topics()
  if "lesson" in tags:
    list_with_lesson.append(repo)
  else:
    list_without_lesson.append(repo)
print("List of Repository with lesson:")   
for repo in list_with_lesson:
  print(repo)
print("\nList of Repository without lesson\n")
for repo in list_without_lesson:
  print(repo)

### KPI 2 > Create Functions to determine & implement parameters describing project activities.in particular

* number of open issues
* number of closed issues
* number of open PRs-0
* number of closed PRs
* number of contributors
* number of starts

In [ ]:
# Declaring a function to count closed_issue_count
def number_of_closed_issues(repo):
    count_all_closed_issue=0
    closed_issues=repo.get_issues(state='closed')
    for issue in closed_issues:
        count_all_closed_issue+=1
    return count_all_closed_issue
    

In [ ]:
# Declaring a function to count open_issue_count
def number_of_open_issues(repo):
    count_all_open_issue=0
    open_issues=repo.get_issues(state='open')
    for issue in open_issues:
        count_all_open_issue+=1
    return count_all_open_issue


In [ ]:
# Declaring a function to count stargazers_count
def stargazers(repo):
    stars=repo.stargazers_count
    return stars


In [ ]:
# Declaring a function to count open_issue_count
def number_of_open_PR(repo):
    count_all_open_PR=0
    pulls=repo.get_pulls(state='open')
    for pr in pulls:
        count_all_open_PR+=1
    return count_all_open_PR

In [ ]:
def number_of_closed_PR(repo):
    count_all_closed_PR=0
    pulls=repo.get_pulls(state='closed')
    for pr in pulls:
        count_all_closed_PR+=1
    return count_all_closed_PR
    

In [ ]:
# Creating a total count of allcategories (Issue_count, Pr_count, Stagazers_count & contributors_count)

def get_final_values(final_repo_list):
    total_closed_Issue_count=0
    total_open_Issue_count=0
    starCount=0
    total_OpenPR_Request=0
    total_ClosedPR_Request=0
    for repo in final_repo_list:
        # closed issues_count
        closedIssues = number_of_closed_issues(repo)
        total_closed_Issue_count += closedIssues
        
        # open issues_count
        openIssues = number_of_open_issues(repo)
        total_open_Issue_count+=openIssues
        
        # Stargazers_count
        stars = stargazers(repo)
        starCount+=stars
        
        # open pr_request_count
        openPR = number_of_open_PR(repo)
        total_OpenPR_Request+=openPR
        
        # closed pr_request_count
        closedPR_Request= number_of_closed_PR(repo)
        total_ClosedPR_Request+=closedPR_Request

    return {'total_closed_Issue_count':total_closed_Issue_count,
            'total_open_Issue_count':total_open_Issue_count,
            'starCount':starCount,
            'total_OpenPR_request': total_OpenPR_Request,
            'total_ClosedPR_request':total_ClosedPR_Request}
    
final_values = get_final_values(final_repo_list)
print(final_values)

###  KPI 3 > Similar to [Carpentries Community Lesson](https://carpentries.org/community-lessons/) we want to list a basic description of each lesson, including
*  lesson title
*  repository
*  Life Cycle Stage
*  Tags
*  Last updated

In [ ]:
# Declaring a function to get_lesson_title,last_update_date,repsoitory_name,Life_cycle_stage of lesson ,tags in each lesson.

import datetime
def lesson_info(repo):
  thisdict={}
  version=""
  date_latest=datetime.datetime(2005, 6, 1)
  name=repo.name
  tags=repo.get_topics()
  title=repo.description
  help_wanted=0
  if "alpha" in tags:
    version="alpha"
  if "pre-alpha" in tags:
    version="pre-alpha"
  if "beta" in tags:
    version="beta"
  if "stable" in tags:
    version="stable"
  open_issues=repo.get_issues(state='open')
  for issue in open_issues:
    labels=issue.get_labels()
    for label in labels:
      if label.name=="help wanted":
        help_wanted+=1
  commits=repo.get_commits()
  for commit in commits:
    time = datetime.datetime.strptime(commit.last_modified,'%a, %d %b %Y %H:%M:%S %Z')
    if (time > date_latest):
      date_latest = time
  thisdict["repo_name"]=name
  thisdict["repo_tags"]=tags
  thisdict["repo_description"]=title
  thisdict["repo_datetime"]=date_latest
  thisdict["repo_version"]=version
  thisdict["repo_label"]=help_wanted
  return thisdict
list_of_topic=[]
for repo in final_repo_list:

  temp_repo_info=lesson_info(repo)
  list_of_topic.append(temp_repo_info)
for topics in list_of_topic:
    print(topics)

KPI 4 > 

*  Total number of lessons in the incubator
*  how many lessons at each state
*  number of lessons with the "help wanted" tag

In [ ]:
# Declaring a function to get number of lesson in the Incubator,count_of_lesson at each state and number_help_wanted_tag.

def lesson_overview(list_of_topic):
  count_all_alpha_state=0
  count_all_pre_alpha_state=0
  count_all_beta_state=0
  count_all_stable_state=0
  total_lesson_count=0
  totallesson_help_wanted=0
  totallesson_updated_last_30_days=0
  today = datetime.datetime.now()
  previous_month = today.replace(month = datetime.datetime.now().month-2)
  for topic in list_of_topic:
    total_lesson_count+=1
    if topic["repo_version"]=="alpha":
      count_all_alpha_state+=1
    if topic["repo_version"]=="pre-alpha":
      count_all_pre_alpha_state+=1
    if topic["repo_version"]=="beta":
      count_all_beta_state+=1
    if topic["repo_version"]=="stable":
      count_all_stable_state+=1
    if topic["repo_label"]!=0:
      totallesson_help_wanted+=1
    if topic["repo_datetime"]>previous_month:
      totallesson_updated_last_30_days+=1
  return {'total_lesson_count':total_lesson_count,'count_all_alpha_state':count_all_alpha_state ,
          'count_all_pre_alpha_state':count_all_pre_alpha_state,'count_all_beta_state':count_all_beta_state,
          'count_all_stable_state':count_all_stable_state,'totallesson_help_wanted':totallesson_help_wanted, 
          'totallesson_updated_last_30_days':totallesson_updated_last_30_days}

final_values=lesson_overview(list_of_topic)
print(final_values)

KPI 5> Analyse collaborative aspects of lesson development

*Total number of people involved in lesson development (also classified by different types of activity: issues, PRs,commits etc 


In [ ]:
# Count of commit and commit_author

cols=["repo","commits","pulls","issues","commit_contributors", "issue_contributors","pulls_contributors","PR_reviewers","collaborators"]#col heading
df=pd.DataFrame(columns=cols)
df['repo']=list_with_lesson
#print(df.head(10))
commitlist=[] # holds the total count of number of commits made in the repositories.
commit_authorlist=[] # contains authors of each commit.
for repo in list_with_lesson: #get count of commits against each repo
    no_of_commit=repo.get_commits().totalCount
    clist=repo.get_commits()# calling the get_commit method of repo object to create the commit list, get_commit() will return a paginated list of commits& store it in clist as list datatype.
    #clist=list(dict.fromkeys(clist))
    commitlist.append(no_of_commit)
    authors=[]
    author_names=[]
    for cauthor in clist:
        if cauthor.commit.author.name not in author_names:
            authors.append(cauthor.commit.author)
            author_names.append(cauthor.commit.author.name)
    commit_authorlist.append(len(author_names))
     
df['commits']=commitlist
df['commit_contributors']=commit_authorlist

In [ ]:
df.style.set_properties(subset=['repo'], **{'width-min': '300px'})

In [ ]:
# Count of pr request and pr_user

pr_requestlist=[]
pr_authorlist=[]
for repo in list_with_lesson:
    no_of_pr=repo.get_pulls(state='open').totalCount
    prlist=repo.get_pulls(state='open')
    pr_requestlist.append(no_of_pr)
    pr_user=[]
    pr_user_name=[]
#     pr_comment=[]
#     pr_merge_user=[]
#     pr_reaction=[]
    for PRuser in prlist:
        if PRuser.user.name not in pr_user_name:
            pr_user.append(PRuser.user)
            pr_user_name.append(PRuser.user.name)
    pr_authorlist.append(len(pr_user_name))
    
    
df['pulls']=pr_requestlist
df['pulls_contributors']=pr_authorlist

In [ ]:
df.style.set_properties(subset=['repo'], **{'width-min': '300px'})

In [ ]:
# Count of  pr_reviewers

pr_review=pulls.get_reviews()
pr_reviewers_user=[]
for review in pr_review:
    if review.user.name not in pr_reviewers_user:
        pr_reviewers_user.append(review.user.name)
        pr_authorlist.append(review.user)
    pr_authorlist.append(len(pr_reviewers_user))


df['PR_reviewers']=pr_reviewers_user

In [ ]:
# to get the pr_comments
pr_comments=repo.ge


In [ ]:
df.style.set_properties(subset=['repo'], **{'width-min': '300px'})

In [ ]:
issue_list=[]
issue_authorlist=[]
for repo in list_with_lesson:
    no_of_issues=repo.get_issues().totalcount
    issuelist=repo.get_issues()
    issue_list.append(no_of_issues)
    issue_user=[]
    issue_comment=[]
    issue_closedby_user=[]
    for issue_user.user not in issuelist:
        if issue_user.append(issue_user.user)
        
    
df['issues']=issue_list
df['issue_contributors']=issue_authorlist